In [1]:
! guardrails hub install hub://guardrails/regex_match --quiet
! guardrails hub install hub://guardrails/valid_range --quiet

Installing hub://guardrails/regex_match...
✅Successfully installed guardrails/regex_match!


Installing hub://guardrails/valid_range...
✅Successfully installed guardrails/valid_range!




In [2]:
from pydantic import BaseModel, Field
from guardrails import Guard, OnFailAction
from guardrails.hub import RegexMatch, ValidRange


class Person(BaseModel):
    name: str
    # Existing way of assigning validators
    age: int = Field(validators=[ValidRange(0, 100, on_fail=OnFailAction.EXCEPTION)])
    is_employed: bool

In [3]:
import json
from guardrails.errors import ValidationError


guard = Guard.for_pydantic(Person)

try:
    guard.validate(json.dumps({"name": "john doe", "age": 101, "is_employed": False}))
except ValidationError as e:
    print(e)

Validation failed for field with errors: Value 101 is greater than 100.


In [5]:
# Now let's add a new validator to the name field

guard.use(
    RegexMatch("^(?:[A-Z][^\\s]*\\s?)+$", on_fail=OnFailAction.EXCEPTION), on="$.name"
)

try:
    guard.validate(json.dumps({"name": "john doe", "age": 30, "is_employed": True}))
except ValidationError as e:
    print(e)

Validation failed for field with errors: Result must match ^(?:[A-Z][^\s]*\s?)+$
